<a href="https://colab.research.google.com/github/yaniamm/MITMO_PLOTNYAGINA/blob/main/LR6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа №6: Машины опорных векторов

In [ ]:
import sklearn
print(sklearn.__version__)

1.2.2


#Загружаем пакеты

In [ ]:
# загрузка пакетов: инструменты --------------------------------------------
# работа с массивами
import numpy as np
# фреймы данных
import pandas as pd
# графики
import matplotlib as mpl
# стили и шаблоны графиков на основе matplotlib
import seaborn as sns
# загрузка файлов по URL
import urllib
# проверка существования файла на диске
from pathlib import Path
# для форматирования результатов с помощью Markdown
from IPython.display import Markdown, display
# перекодировка категориальных переменных
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
# хи-квадрат тест на независимость по таблице сопряжённости
from scipy.stats import chi2_contingency
# для таймера
import time
# загрузка пакетов: данные -------------------------------------------------
from sklearn import datasets
# загрузка пакетов: модели -------------------------------------------------
# дерево классификации
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
# перекрёстная проверка и метод проверочной выборки
from sklearn.model_selection import cross_val_score, train_test_split
# для перекрёстной проверки и сеточного поиска
from sklearn.model_selection import KFold, GridSearchCV
# бэггинг
from sklearn.ensemble import BaggingClassifier
# случайный лес
from sklearn.ensemble import RandomForestClassifier
# бустинг
from sklearn.ensemble import GradientBoostingClassifier
# сводка по точности классификации
from sklearn.metrics import classification_report

In [ ]:
# константы
# ядро для генератора случайных чисел
my_seed = 18
# создаём псевдоним для короткого обращения к графикам
plt = mpl.pyplot
# настройка стиля и отображения графиков
# примеры стилей и шаблонов графиков:
# http://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html
mpl.style.use('seaborn-whitegrid')
sns.set_palette("Set2")
# раскомментируйте следующую строку, чтобы посмотреть палитру

<ipython-input-3-0dd040cdffd9>:9: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  mpl.style.use('seaborn-whitegrid')


In [ ]:
# функция форматирования результатов с использованием Markdown
def printmd(string):
 display(Markdown(string))

# функции для попарной конкатенации элементов двух списков
concat_func_md = lambda x, y: '`' + str(x) + "`:&ensp;&ensp;&ensp;&ensp;" + str(y)
concat_func = lambda x, y: str(x) + ' ' * 4 + str(y)
# функция, которая строит график важности признаков в модели случайного леса
# источник: https://www.analyseup.com/learn-python-for-data-science/python-rand
def plot_feature_importance(importance, names, model_type):
 #Create arrays from feature importance and feature names
 feature_importance = np.array(importance)
 feature_names = np.array(names)
 #Create a DataFrame using a Dictionary
 data={'feature_names':feature_names,'feature_importance':feature_importance}
 fi_df = pd.DataFrame(data)
 #Sort the DataFrame in order decreasing feature importance
 fi_df.sort_values(by=['feature_importance'], ascending=False,
 inplace=True)
 #Define size of bar plot
 plt.figure(figsize=(10,8))
 #Plot Searborn bar chart
 sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
 #Add chart labels
 plt.title('Важность признаков в модели: ' + model_type)
 plt.xlabel('Важность признака')
 plt.ylabel('')

#Загружаем данные
Загружаем данные во фрейм и выясняем их размерность. Смотрим первые строки таблицы

In [ ]:
fileURL = "https://raw.githubusercontent.com/ania607/ML/main/data/default_of_credit_card_clients.csv"
DF_all = pd.read_csv(fileURL)
# загружаем таблицу и превращаем её во фрейм
# выясняем размерность фрейма
print('Число строк и столбцов в наборе данных:\n', DF_all.shape)

Число строк и столбцов в наборе данных:
 (30000, 24)


In [ ]:
# первые строки
DF_all.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,Y
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [ ]:
# типы столбцов
DF_all.dtypes

LIMIT_BAL    int64
SEX          int64
EDUCATION    int64
MARRIAGE     int64
AGE          int64
PAY_0        int64
PAY_2        int64
PAY_3        int64
PAY_4        int64
PAY_5        int64
PAY_6        int64
BILL_AMT1    int64
BILL_AMT2    int64
BILL_AMT3    int64
BILL_AMT4    int64
BILL_AMT5    int64
BILL_AMT6    int64
PAY_AMT1     int64
PAY_AMT2     int64
PAY_AMT3     int64
PAY_AMT4     int64
PAY_AMT5     int64
PAY_AMT6     int64
Y            int64
dtype: object

все столбцы оказались количественными

отложим 10% данных для моделирования

In [ ]:
# наблюдения для моделирования
DF = DF_all.sample(frac=0.9, random_state=my_seed)
# отложенные наблюдения
DF_predict = DF_all.drop(DF.index)

#Преобразование исходных данных и построение моделей
Как указано в справочнике к данным, объясняющие переменные уже являются производными
от описательных статистик 10 показателей, рассчитанных по визуализации клеток опухоли. Все
эти переменные количественные, и как видно из отчёта ниже, их значения неотрицательные, а
разброс различается

In [ ]:
DF.iloc[:, 2:].describe()

,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,Y
count,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000,...,27000.000000,27000.000000,27000.000000,27000.000000,2.700000e+04,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000
mean,1.855222,1.551037,35.496370,-0.017222,-0.134519,-0.164000,-0.221185,-0.266667,-0.290370,50962.328444,...,43073.211074,40124.493852,38698.586926,5666.423704,5.893804e+03,5158.335815,4799.184333,4783.138852,5205.597222,0.221963
std,0.792878,0.522527,9.215327,1.124578,1.196830,1.195814,1.167499,1.130147,1.150053,73119.084597,...,63983.036553,60522.659119,59264.303747,16734.126862,2.355366e+04,16752.107225,15661.876395,15352.646875,17726.484723,0.415574
min,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-165580.000000,...,-170000.000000,-81334.000000,-209051.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,3563.750000,...,2297.750000,1764.750000,1241.500000,1000.000000,8.237500e+02,390.000000,299.000000,244.000000,102.000000,0.000000
50%,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22320.000000,...,19001.000000,18064.000000,16934.500000,2100.000000,2.004000e+03,1800.000000,1500.000000,1500.000000,1500.000000,0.000000
75%,2.000000,2.000000,42.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,66635.500000,...,53621.750000,49812.500000,48863.500000,5006.000000,5.000000e+03,4500.000000,4013.250000,4008.250000,4000.000000,0.000000
max,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,653062.000000,...,706864.000000,823540.000000,699944.000000,873552.000000,1.684259e+06,889043.000000,621000.000000,426529.000000,528666.000000,1.000000


В качестве альтернативных моделей рассмотрим SVM с различными вариантами ядер и
логистическую регрессию. Причём предварительно преобразуем пространство исходных
показателей с помощью метода главных компонент.

#Стандартизация и переход к главным компонентам

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# стандартизация
sc = StandardScaler()
X_train_std = sc.fit_transform(DF.iloc[:, 2:].values)
# оцениваем объяснённую главными компонентами дисперсию
pca = PCA()
X_train_pca = pca.fit_transform(X_train_std)
# считаем доли объяснённой дисперсии
frac_var_expl = pca.explained_variance_ratio_
print('Доли объяснённой дисперсии по компонентам в PLS:\n',
 np.around(frac_var_expl, 3),
 '\nОбщая сумма долей:', np.around(sum(frac_var_expl), 3))

Доли объяснённой дисперсии по компонентам в PLS:
 [0.297 0.175 0.071 0.067 0.044 0.041 0.04  0.04  0.039 0.035 0.033 0.028
 0.027 0.018 0.012 0.011 0.009 0.006 0.003 0.002 0.001 0.001] 
Общая сумма долей: 1.0


Таким образом, первые две главные компоненты объясняют 63% разброса 22 объясняющих
переменных.
Теперь объединим функции-преобразователи и оценщики в конвейер с помощью Pipeline
и оценим точность логистической регрессии с помощью перекрёстной проверки.


#Модель логистической регрессии с перекрёстной проверкой

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# данные для обучения моделей
X_train = DF.iloc[:, 2:]
y_train = DF.iloc[:, 1]

# объединяем в конвейер шкалирование, ГК с 2 компонентами и логит
pipe_lr = make_pipeline(StandardScaler(), PCA(n_components=2), LogisticRegression(random_state=my_seed, solver='lbfgs'))
# будем сохранять точность моделей в один массив:
score = list()
score_models = list()
# считаем точность с перекрёстной проверкой, показатель Acc
cv = cross_val_score(estimator=pipe_lr, X=X_train, y=y_train, cv=5, scoring='accuracy')

# записываем точность
score.append(np.around(np.mean(cv), 3))
score_models.append('sc_pca_logit')
print('Acc с перекрёстной проверкой',
 '\nдля модели', score_models[0], ':', score[0])

Acc с перекрёстной проверкой 
для модели sc_pca_logit : 0.605


#SVM с перекрёстной проверкой
Построим несколько вариантов модели SVM с различными ядерными функциями
Так как объем данных очень большой не предоставляется возможным запустить сеточный поиск, поэтому в работе было выполнена только logit регрессия

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

pipe_svc = make_pipeline(StandardScaler(), SVC(random_state=my_seed))

# настроим параметры SVM с помощью сеточного поиска
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_grid = [{'svc__C': param_range, 'svc__kernel': ['linear']},
              {'svc__C': param_range, 'svc__gamma': param_range, 'svc__kernel': ['rbf']},
              {'svc__C': param_range, 'svc__gamma': param_range, 'svc__degree': [2, 3], 'svc__kernel': ['poly']}]

# разбиения для перекрёстной проверки
kfold = KFold(n_splits=5, random_state=my_seed, shuffle=True)

gs = GridSearchCV(estimator=pipe_svc, param_grid=param_grid,
                  scoring='accuracy', refit=True, cv=kfold, n_jobs=-1)

# таймер
tic = time.perf_counter()
# запускаем сеточный поиск
gs = gs.fit(X_train, y_train)
# таймер
toc = time.perf_counter()
print("Сеточный поиск занял {:.2f} секунд".format(toc - tic))

In [ ]:
# точность лучшей модели
np.around(gs.best_score_, 3)

In [ ]:
# параметры лучшей модели
# * ядерная функция
gs.best_estimator_.get_params()['svc__kernel']

In [ ]:
# * параметр регуляризации
gs.best_estimator_.get_params()['svc__C']


In [ ]:
# * коэффициент ядерной функции (для ядер 'rbf', 'poly' и 'sigmoid')
gs.best_estimator_.get_params()['svc__gamma']

In [ ]:
# * степень полинома (для ядра 'poly')
gs.best_estimator_.get_params()['svc__degree']

In [ ]:
# сводка по точности моделей
pd.DataFrame({'Модель' : score_models, 'Acc' : score})

In [ ]:
# прогноз с помощью лучшей модели ансамбля с SVC
y_hat = gs.best_estimator_.predict(X=DF_predict.loc[:, 2:])
# точность
# характеристики точности
print(classification_report(DF_predict.loc[:, 1], y_hat))